In [39]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('student.csv')
df.head()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,grade
0,18,4,4,2,2,0,4,3,4,1,1,3,4,1
1,17,1,1,1,2,0,5,3,3,1,1,3,2,1
2,15,1,1,1,2,0,4,3,2,2,3,3,6,1
3,15,4,2,1,3,0,3,2,2,1,1,5,0,2
4,16,3,3,1,2,0,4,3,2,1,2,5,0,1


In [40]:
# Обучим модель сначала без стандартизации данных, а потом с ней.
# И посмотрим на разницу в качестве обучения
from sklearn.linear_model import LogisticRegression


X_train, X_test, y_train, y_test = train_test_split(df.drop('grade', axis=1),
                                                    df['grade'],
                                                    test_size=0.2,
                                                    random_state=10)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
logreg.coef_

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array([[ 0.18371065,  0.46279065,  0.02488919, -0.14084531,  0.38231845,
        -1.33101237,  0.00314899,  0.05123544, -0.01806714, -0.66531539,
         0.02035311, -0.13587083, -0.05586447]])

In [41]:
logreg.feature_names_in_

array(['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures',
       'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health',
       'absences'], dtype=object)

In [42]:
from sklearn.metrics import accuracy_score


accuracy_score(logreg.predict(X_test), y_test)

0.7846153846153846

In [43]:
# Проверим, что модель не переобучилась
accuracy_score(logreg.predict(X_train), y_train)

0.7514450867052023

In [44]:
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [45]:
pd.DataFrame(X_test_scaled, columns=[i for i in df.columns if i != 'grade']).describe()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences
count,130.000000,130.000000,130.000000,130.000000,130.000000,130.000000,130.000000,130.000000,130.000000,130.000000,130.000000,130.000000,130.000000
mean,-0.022182,0.153035,-0.042504,-0.148906,-0.023272,0.071271,0.020505,0.114484,0.024606,0.178292,0.172958,0.094968,0.144082
std,1.105554,0.977303,0.998206,0.844029,1.051207,1.261951,1.056953,0.972767,1.073331,1.115407,1.106381,0.991395,1.150700
min,-1.467708,-2.180016,-2.105134,-0.768601,-1.138322,-0.381328,-3.098001,-2.042433,-1.881351,-0.521816,-0.986612,-1.732888,-0.786004
25%,-0.628787,-0.421775,-1.196178,-0.768601,-1.138322,-0.381328,0.077501,-0.147844,-1.018025,-0.521816,-0.986612,-0.351369,-0.786004
50%,0.210134,0.457346,-0.287223,-0.768601,0.079800,-0.381328,0.077501,-0.147844,-0.154700,-0.521816,-0.189024,0.339391,-0.340659
75%,1.049055,1.336467,0.621732,0.530760,0.079800,-0.381328,1.136002,0.799451,0.708625,0.588112,0.608564,1.030150,0.550031
max,4.404740,1.336467,1.530688,3.129482,2.516044,4.967577,1.136002,1.746746,1.571951,3.917894,2.203740,1.030150,6.339516


In [46]:
X_train_scaled = pd.DataFrame(X_train_scaled, columns=[i for i in df.columns if i != 'grade'])
X_test_scaled = pd.DataFrame(X_test_scaled, columns=[i for i in df.columns if i != 'grade'])
logreg = LogisticRegression()
logreg.fit(X_train_scaled, y_train)
y_pred = logreg.predict(X_test_scaled)
accuracy_score(y_test, y_pred)

0.7846153846153846

In [47]:
accuracy_score(y_train, logreg.predict(X_train_scaled))

0.7572254335260116

Чёт всё хорошо и в этом случае стандартизация не повлияла на качество модели :)

In [52]:
# Обучим модель с регуляризацией и без
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline


accuracies = []
for i in range(1, 7):
    model = make_pipeline(
        PolynomialFeatures(degree=i),
        StandardScaler(),
        LogisticRegression(penalty=None)
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracies.append(f'Degree: {i}, accuracy: {accuracy_score(y_test, y_pred)}')

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [53]:
for i in range(len(accuracies)):
    print(accuracies[i])

Degree: 1, accuracy: 0.7769230769230769
Degree: 2, accuracy: 0.7230769230769231
Degree: 3, accuracy: 0.6692307692307692
Degree: 4, accuracy: 0.6846153846153846
Degree: 5, accuracy: 0.7076923076923077
Degree: 6, accuracy: 0.6692307692307692


In [54]:
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.6692307692307692

In [55]:
accuracy_score(y_train, model.predict(X_train))

0.9633911368015414

In [56]:
# Добавим регуляризацию и, по совету сообщений увеличим max_iter
C = [0.0001, 0.001, 0.01, 0.1, 1]
accuracy = []  # Degree, C, accuracy

for i in range(1, 7):
    for c in C:
        model = make_pipeline(
            PolynomialFeatures(degree=i),
            StandardScaler(),
            LogisticRegression(max_iter=600, C=c)
        )
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy.append((i, c, accuracy_score(y_pred, y_test)))

df = pd.DataFrame(accuracy, columns=['Degree', 'C', 'accuracy_score'])
df[df['accuracy_score'] == df['accuracy_score'].max()]

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

,Degree,C,accuracy_score
9,2,1.0,0.830769


In [65]:
df.groupby('Degree').agg({'accuracy_score': np.max})

<ipython-input-65-47d531507df6>:1: FutureWarning: The provided callable <function max at 0x7efe4f6b7490> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df.groupby('Degree').agg({'accuracy_score': np.max})


,accuracy_score
Degree,
1,0.815385
2,0.830769
3,0.815385
4,0.807692
5,0.800000
6,0.800000


In [69]:
model = make_pipeline(
            PolynomialFeatures(degree=2),
            StandardScaler(),
            LogisticRegression(max_iter=600, C=1.0)
)
model.fit(X_train, y_train)
print(accuracy_score(y_train, model.predict(X_train)))
accuracy_score(y_test, model.predict(X_test))

0.791907514450867


0.8307692307692308

Теперь видим, что модель не переобучилась и показывает более хорошие результаты на тестовой выборке